In [2]:
import numpy as np
import cv2
import imutils
import easyocr
import requests
import re
def get_book_info(isbn):
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'items' in data and len(data['items']) > 0:
            book_info = data['items'][0]['volumeInfo']
            title = book_info.get('title', 'Title not available')
            authors = ", ".join(book_info.get('authors', ['Author not available']))
            description = book_info.get('description', 'Description not available')
            image_link = book_info.get('imageLinks', {}).get('thumbnail', 'Image not available')
            return title, authors, description, image_link
        else:
            return "Book not found", "", "", ""
    else:
        return "Error: Unable to retrieve book information", "", "", ""
def Book_Title(image):
    # Resize the image - change width to 500
    image = imutils.resize(image, width=500)

    # Display the original image
    cv2.imshow("Original Image", image)
    cv2.waitKey(0)

    # RGB to Gray scale conversion
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Gray Image", gray)
    cv2.waitKey(0)

    # Noise removal with iterative bilateral filter(removes noise while preserving edges)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    cv2.imshow("Bilateral Filter", gray)
    cv2.waitKey(0)

    # Find Edges of the grayscale image
    edged = cv2.Canny(gray, 170, 200)
    cv2.imshow("Canny Edges", edged)
    cv2.waitKey(0)

    # Find contours based on Edges
    cnts, new  = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Create copy of original image to draw all contours
    img1 = image.copy()
    cv2.drawContours(img1, cnts, -1, (0,255,0), 3)
    cv2.imshow("All Contours", img1)
    cv2.waitKey(0)
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
    img2 = image.copy()
    cv2.drawContours(img2, cnts, -1, (0,255,0), 3)
    #cv2.imshow("Top 30 Contours", img2)
    cv2.waitKey(0)

    reader = easyocr.Reader(['en'])
    result = reader.readtext(img2)
#     for i in range(len(result)):
#         print(result[i][-2])
#     print(result[1][-2])
    # Assuming 'result' is your list of lists
    result_one_dimensional = [text_element[-2] for text_element in result]

    # Alternatively, you can achieve the same using loops:
    # result_one_dimensional = []
    # for text_element in result:
    #     result_one_dimensional.append(text_element[-2])
    # Print the resulting one-dimensional list
#     print(result_one_dimensional)
    # Assuming 'result' is your list of strings
    result_string = ' '.join(result_one_dimensional)
    # Print the resulting string
#     print(result_string)
    result_string = result_string.replace(" ", "")
    result_string = "".join(result_string.split())
#     print(result_string)
    index_of_N = result_string.find("N")

    # Check if 'N' is found in the string
    if index_of_N != -1:
        # Extract substring after 'N' and take the next 17 characters
        substring = result_string[index_of_N + 1:index_of_N + 18]
#         print(substring)
    else:
        print("Character 'N' not found in the string.")

    return substring
image = cv2.imread('test1.jpg')
isbn=Book_Title(image)
print("isbn:",isbn)
isbn=str(isbn)
title, authors, description, image_link = get_book_info(isbn)
print("Book Title:", title)
print("Authors:", authors)
print("Description:", description)
# print("Image Link:", image_link)
cv2.waitKey(0) #Wait for user input before closing the images displayed
cv2.destroyAllWindows()  # Close all OpenCV windows

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


isbn: 978-93-325-5001-8
Book Title: Modern Operating Systems
Authors: Andrew S. Tanenbaum
Description: Description not available
